In [ ]:
import sys
import math
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# import seaborn as sns
import requests
import json

In [ ]:
with open('taxee_creds.json','r') as f:
    creds = json.load(f)

In [ ]:
r = requests.get('https://taxee.io/api/v2/federal/2021', headers =creds )

In [ ]:
fed = r.json()

In [ ]:
fed

In [ ]:
with open('fed.json', 'w') as f:
    json.dump(r.json(), f)

In [ ]:
with open('fed.json', 'r') as f:
    fed = json.load(f)

In [ ]:
data = fed['single']
deduction = data['deductions'][0]['deduction_amount']
brackets = [i['bracket'] for i in data['income_tax_brackets']]
tax =  [i['marginal_rate'] for i in data['income_tax_brackets']]

In [ ]:
print(deduction)
print(brackets)
print(len(brackets))
print(tax)
print(len(tax))

In [ ]:
fed

In [ ]:
def tax(tax_dict, income, filing_status):
    
    
    data = tax_dict[filing_status]
    try:
        deduction = data['deductions'][0]['deduction_amount']
    except:
        deduction = 0
    brackets = [i['bracket'] for i in data['income_tax_brackets']]
    brackets.reverse()
    print(brackets)
    tax = [i['marginal_rate']/100 for i in data['income_tax_brackets']]
    tax.reverse()
    print(tax)
    taxable_income = income - deduction
    
    tax_out = 0
    inc_remain = taxable_income
    for b,t in zip(brackets,tax):
        if inc_remain > b:
            taxable = inc_remain - b
            to = taxable * t
            tax_out += to
            inc_remain = b


    return tax_out

In [ ]:
tax(fed, 200000, 'single')

In [ ]:
states="""Alabama	AL
Alaska	AK
Arizona	AZ
Arkansas	AR
California	CA
Colorado	CO
Connecticut	CT
Delaware	DE
Florida	FL
Georgia	GA
Hawaii	HI
Idaho	ID
Illinois	IL
Indiana	IN
Iowa	IA
Kansas	KS
Kentucky	KY
Louisiana	LA
Maine	ME
Maryland	MD
Massachusetts	MA
Michigan	MI
Minnesota	MN
Mississippi	MS
Missouri	MO
Montana	MT
Nebraska	NE
Nevada	NV
New Hampshire	NH
New Jersey	NJ
New Mexico	NM
New York	NY
North Carolina	NC
North Dakota	ND
Ohio	OH
Oklahoma	OK
Oregon	OR
Pennsylvania	PA
Rhode Island	RI
South Carolina	SC
South Dakota	SD
Tennessee	TN
Texas	TX
Utah	UT
Vermont	VT
Virginia	VA
Washington	WA
West Virginia	WV
Wisconsin	WI
Wyoming	WY"""

In [ ]:
state_dict = {}
for i in states.split('\n'):
    o=i.split('\t')
    state_dict[o[0]] = o[1]

In [ ]:
year = '2020'
for v in state_dict.values():
    r = requests.get(f'https://taxee.io/api/v2/state/{year}/{v}', headers =creds )
    with open(f'state_tax/{year}/{v}.json', 'w') as f:
        json.dump(r.json(), f)

In [ ]:
def read_state_tax(state,year):
    with open(f'state_tax/{year}/{state}.json', 'r') as f:
        return json.load(f)

In [ ]:
mi=read_state_tax('MI','2019')
mi

In [ ]:
tax(mi, 200000,'single')

In [ ]:
year = '2020'
r = requests.get(f'https://taxee.io/api/v2/federal/{year}', headers =creds )
with open(f'federal_tax/{year}/fed.json', 'w') as f:
    json.dump(r.json(), f)